In [ ]:
import cv2
from ultralytics import YOLO

# ==============================
# CONFIG
# ==============================
STREAM_URL="http://172.18.230.94:8080/video"
MODEL_PATH="best.pt"
CONF_THRES=0.4

# ==============================
# LOAD MODEL
# ==============================
model=YOLO(MODEL_PATH)
print("✅ YOLO model loaded:",model.names)

# ==============================
# VIDEO STREAM
# ==============================
cap=cv2.VideoCapture(STREAM_URL)

if not cap.isOpened():
    print("❌ Cannot open IP Webcam stream")
    exit()

print("✅ Stream opened")

# ==============================
# MAIN LOOP
# ==============================
while True:
    ret,frame=cap.read()
    if not ret:
        print("❌ Frame grab failed")
        continue

    # ---------- YOLO INFERENCE ----------
    results=model(frame,conf=CONF_THRES,verbose=False)

    for r in results:
        if r.boxes is None:
            continue

        for box in r.boxes:
            x1,y1,x2,y2=map(int,box.xyxy[0])
            conf=float(box.conf[0])

            # bounding box
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)

            # label
            cv2.putText(
                frame,
                f"char {conf:.2f}",
                (x1,y1-8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0,255,0),
                2
            )

    # ------------------------------------
    cv2.imshow("IP Webcam + YOLO",frame)

    if cv2.waitKey(1)&0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
